In [ ]:
from gs_quant.session import GsSession

GsSession.use()

In [ ]:
from gs_quant.instrument import OptionStyle
from gs_quant.backtests.strategy import Strategy
from gs_quant.backtests.triggers import *
from gs_quant.backtests.actions import *
from gs_quant.backtests.equity_vol_engine import *
from datetime import datetime, date

import gs_quant.risk as risk

In [ ]:
# Initialize session
from gs_quant.session import GsSession

GsSession.use(client_id=None, client_secret=None, scopes=('run_analytics',))

In [ ]:
# Define backtest dates
start_date = date(2024, 4, 1)
end_date = datetime.today().date()

In [ ]:
# Define instruments for strategy
# Portfolio of two eq options
call = EqOption(
    '.STOXX50E',
    expiration_date='1m',
    strike_price='ATM',
    option_type=OptionType.Call,
    option_style=OptionStyle.European,
    name='call',
)
put = EqOption(
    '.STOXX50E',
    expiration_date='1m',
    strike_price='ATM',
    option_type=OptionType.Put,
    option_style=OptionStyle.European,
    name='put',
)
portfolio = Portfolio(name='portfolio', priceables=[call, put])

### Entering a position monthly

In [ ]:
# Trade the position monthly without any scaling
trade_action = AddTradeAction(priceables=portfolio, trade_duration='1m', name='act')
trade_trigger = PeriodicTrigger(
    trigger_requirements=PeriodicTriggerRequirements(start_date=start_date, end_date=end_date, frequency='1m'),
    actions=trade_action,
)

strategy = Strategy(None, trade_trigger)

# Run backtest daily
engine = EquityVolEngine
backtest = engine.run_backtest(strategy, start=start_date, end=end_date)

In [ ]:
pnl = backtest.get_measure_series(FlowVolBacktestMeasure.PNL)
pnl.plot(legend=True, label='PNL')

### Scaling quantity

In [ ]:
# Scale the position quantity (no. of options) by 2
trade_action_scaled = AddScaledTradeAction(
    priceables=portfolio,
    trade_duration='1m',
    scaling_type=ScalingActionType.size,
    scaling_risk=None,
    scaling_level=2,
    name='scaled_act',
)
trade_trigger_scaled = PeriodicTrigger(
    trigger_requirements=PeriodicTriggerRequirements(start_date=start_date, end_date=end_date, frequency='1m'),
    actions=trade_action_scaled,
)

strategy = Strategy(None, trade_trigger_scaled)

# Run backtest daily
engine = EquityVolEngine
backtest = engine.run_backtest(strategy, start=start_date, end=end_date)

In [ ]:
# note the performance is twice the above graph
pnl = backtest.get_measure_series(FlowVolBacktestMeasure.PNL)
pnl.plot(legend=True, label='PNL')

### Scaling a risk measure

In [ ]:
# Initial and final Vega of the portfolio
with PricingContext(start_date):
    start_vega = portfolio.calc(risk.EqVega)

with PricingContext(end_date):
    end_vega = portfolio.calc(risk.EqVega)

print(start_vega.aggregate())
print(end_vega.aggregate())

In [ ]:
# Scale the position vega to 100
trade_action_scaled = AddScaledTradeAction(
    priceables=portfolio,
    trade_duration='1m',
    scaling_type=ScalingActionType.risk_measure,
    scaling_risk=risk.EqVega,
    scaling_level=100,
    name='scaled_vega_act',
)
trade_trigger_scaled = PeriodicTrigger(
    trigger_requirements=PeriodicTriggerRequirements(start_date=start_date, end_date=end_date, frequency='1m'),
    actions=trade_action_scaled,
)

strategy = Strategy(None, trade_trigger_scaled)

# Run backtest daily
engine = EquityVolEngine
backtest = engine.run_backtest(strategy, start=start_date, end=end_date)

In [ ]:
# note the scaling should be approx the 100/vega
pnl = backtest.get_measure_series(FlowVolBacktestMeasure.PNL)
pnl.plot(legend=True, label='PNL')

### NAV Trading

In [ ]:
# Create a NAV-based action that starts with 100 available to spend
start_date = date(2024, 1, 1)
end_date = datetime.today().date()

trade_action_scaled = EnterPositionQuantityScaledAction(
    priceables=portfolio,
    trade_duration='1m',
    trade_quantity=100,
    trade_quantity_type=BacktestTradingQuantityType.NAV,
    name='nav_act',
)
trade_trigger_scaled = PeriodicTrigger(
    trigger_requirements=PeriodicTriggerRequirements(start_date=start_date, end_date=end_date, frequency='1m'),
    actions=trade_action_scaled,
)

strategy = Strategy(None, trade_trigger_scaled)

# Run backtest daily
engine = EquityVolEngine
backtest = engine.run_backtest(strategy, start=start_date, end=end_date)

In [ ]:
pnl = backtest.get_measure_series(FlowVolBacktestMeasure.PNL)
pnl.plot(legend=True, label='PNL')